In [1]:
import pandas as pd
import numpy as np
import os as os
#import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import classification_report
from joblib import dump, load
import acessos as ac
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:

query = """select 
id_cliente
, dt_resposta as ingestion_date
, ds_avaliacao from 
cx_curated_zone.indecx_nps
where ds_avaliacao is not null"""

df = ac.df_athena_q('flavia-costa', query)

In [4]:
df.head()

,id_cliente,ingestion_date,ds_avaliacao
0,None,2023-11-07 17:51:27,Atendimento top
1,None,2023-11-07 19:57:13,Muito satisfeito com tudo
2,None,2023-11-07 22:23:30,Muito bom ; só queria ter cartão de crédito
3,None,2023-11-08 00:36:06,Muito top
4,None,2023-11-08 05:30:37,E muito prático e fácil de acessar


In [5]:
len(df)

361490

## Aplicando modelo sentimento

In [42]:
from sklearn.neural_network import MLPClassifier


vetorizador = load('vetorizador_sentimento.joblib') 

In [43]:
X = vetorizador.transform(df['ds_avaliacao'])

In [44]:
clf_sentiment = load('modelo_sentimento.joblib') 

In [45]:
df['sent_predito_1'] = clf_sentiment.predict(X)

### Palavras chave 

In [46]:
chave_negativo = ['falta de respeito', 
'judicial',
'procon',
'jurídico',
'porra',
'merda', 
'palhaçada',
'carai',
'caralho',
'bosta',
'vocês demoram',
'voces demoram',
'vcs demoram',
'horrivel',
'banco ruim',
'péssimo',
'danos morais',
'pelo amor de deus',
'eu preciso resolver',
'atender??',
'me roubando']


não fui respondid
não tive resposta
não resolveu o meu problema
meu problema ainda não foi resolvido
não ajudou

In [47]:
chave_positivo = [
'ótimo',
'otimo',
'perfeito',
'muito obrigada',
'excelente',
'muito satisfeit',
'bom atendimento',
'ótimo dia',
'muito top',
'atendimento top',
'maravilha',
'deus abençoe']

Acrescentar:

Atendimento maravilhoso
atendimento nota 10
ótimo atendimento
atendimento excelente

In [48]:
df['sent_predito_1'].value_counts()

1    171964
2    116168
0     73358
Name: sent_predito_1, dtype: int64

In [49]:
def marca_negativo(x):
    for i in chave_negativo:
        if (x.find(i) != -1):
            return 1
        else:
            return 0
   
df['flag_negativo'] = df['ds_avaliacao'].str.lower().apply(marca_negativo)

In [64]:
def marca_positivo(x):
    for i in chave_positivo:
        if (x.find(i) != -1):
            return 1
        else:
            return 0
   
df['flag_positivo'] = df['ds_avaliacao'].str.lower().apply(marca_positivo)

Para a pesquisa do CSAT temos uma tendencia maior para comentários positivos portanto podemos alterar a regra, que está diferente para análise de sentimento do chat

In [51]:
df['sent_predito_final'] = np.where((df['flag_positivo'] == 1),  2,
                                 np.where(df['flag_negativo'] == 1,  0,
                                          df['sent_predito_1']))

df['sent_predito_final'].value_counts()

1    167373
2    127452
0     66665
Name: sent_predito_final, dtype: int64

### Modelo do tipo de mensagem

Gostaria de saber
Como faço
como faço
problema
não reconheço
não fiz essa compra
não reconhecida
fui roubad
fui assaltad
limite não voltou
não consegui pagar
pix não chegou
compra que não fiz
nunca comprei
quero alterar meu
quero resolver
queria que fosse resolvido
preciso resolver
quero aumentar o meu limite
queria o aumento do meu cartao
fiquei na dúvida
paguei mas não aparece
prefiro parcelar

In [52]:
clf_tipo_msg = load('modelo_tipo_mensagem.joblib')

In [53]:
df['tipo_msg_predito'] = clf_tipo_msg.predict(X)

In [54]:
df['tipo_msg_predito'].value_counts()

2    215155
0     84450
1     61885
Name: tipo_msg_predito, dtype: int64

In [55]:
df['sentimento_predito'] = np.where(df['sent_predito_final'] == 0,  '0. Negativo', 
                           np.where(df['sent_predito_final'] == 2,  '2. Positivo', '1. Neutro'))

In [56]:
df['tipo_msg_predito'] = np.where(df['tipo_msg_predito'] == 0,  '0. Problema', 
                           np.where(df['tipo_msg_predito'] == 1,  '1. Solicitacao', '2. Duvida'))

### Avaliação final e exportação

In [57]:
df.head()

,id_cliente,ingestion_date,ds_avaliacao,sent_predito_1,flag_negativo,flag_positivo,sent_predito_final,tipo_msg_predito,sentimento_predito
0,None,2023-11-07 17:51:27,Atendimento top,0,0,0,0,1. Solicitacao,0. Negativo
1,None,2023-11-07 19:57:13,Muito satisfeito com tudo,2,0,0,2,2. Duvida,2. Positivo
2,None,2023-11-07 22:23:30,Muito bom ; só queria ter cartão de crédito,2,0,0,2,2. Duvida,2. Positivo
3,None,2023-11-08 00:36:06,Muito top,2,0,0,2,2. Duvida,2. Positivo
4,None,2023-11-08 05:30:37,E muito prático e fácil de acessar,1,0,0,1,0. Problema,1. Neutro


In [70]:
avaliacoes_negativas = df.query('sentimento_predito == "0. Negativo"')[['ds_avaliacao', 'ingestion_date']].groupby('ds_avaliacao').count().sort_values(by='ingestion_date', ascending = False)

In [71]:
avaliacoes_negativas.head(20)

,ingestion_date
ds_avaliacao,
Gostei,1020
Nada,528
O atendimento,511
Atendimento,439
Atendimento rápido,286
Gostei do atendimento,273
Agilidade no atendimento,154
Rapidez no atendimento,152
Gostei muito do atendimento,148


In [72]:
avaliacoes_positivas = df.query('sentimento_predito == "2. Positivo"')[['ds_avaliacao', 'ingestion_date']].groupby('ds_avaliacao').count().sort_values(by='ingestion_date', ascending = False)

In [73]:
avaliacoes_positivas.head(20)

,ingestion_date
ds_avaliacao,
Muito bom,9315
Ótimo,4701
Bom,2782
Obrigado,1314
Excelente,1258
Ótimo atendimento,1196
Gostei muito,1064
Ok,981
Bom atendimento,848


In [75]:
df.sample(100).to_csv('sentimento_nps.csv', index = False, sep='|')

In [61]:
""" area = 'cx'
name = 'analise_sentimento_teste.csv'
filename = f'{area}.{name}'

df[['cpf_customer', 'mes_chat', 'dt_envio_mensagem',
       'ds_message', 'sent_predito_1','tipo_msg_predito', 'sentimento_predito']].to_csv(filename, index=False)

os.system(f'aws s3 cp {filename} s3://data-sandbox-zone-will-prod/upload/') """

" area = 'cx'\nname = 'analise_sentimento_teste.csv'\nfilename = f'{area}.{name}'\n\ndf[['cpf_customer', 'mes_chat', 'dt_envio_mensagem',\n       'ds_message', 'sent_predito_1','tipo_msg_predito', 'sentimento_predito']].to_csv(filename, index=False)\n\nos.system(f'aws s3 cp {filename} s3://data-sandbox-zone-will-prod/upload/') "